## Tutorial 2: Xenium datasets preprocess demo
#### 1. Download the raw data from 10x Genoimcs ：Metadata:https://www.10xgenomics.com/datasets/human-liver-data-xenium-human-multi-tissue-and-cancer-panel-1-standard; Alignment file: https://cf.10xgenomics.com/samples/xenium/1.9.0/Xenium_V1_hLiver_nondiseased_section_FFPE/Xenium_V1_hLiver_nondiseased_section_FFPE_he_imagealignment.csv OME tif: https://cf.10xgenomics.com/samples/xenium/1.9.0/Xenium_V1_hLiver_nondiseased_section_FFPE/Xenium_V1_hLiver_nondiseased_section_FFPE_he_image.ome.tif
#### 2. Preprocess-data format
- `re_image.tif`：Raw histology image
- `pseudo_st.csv`：Gene count matrix。
  - Row 1: Gene names.
  - Row 2 and after: Each row is a spot.
    - Column 1: Spot ID.
    - Column 2 and after: Each column is a gene.
- `pseudo_locs.csv`：Spot locations
  - Row 1: Header
  - Row 2 and after: Each row is a spot. Must match rows in `pseudo_st.csv`
    - Column 1: Spot ID
    - Column 2: x-coordinate (horizontal axis). Must be in the same space as axis-1 (column) of the array indices of pixels in `re_image.tif`.
    - Column 3: y-coordinate (vertical axis). Must be in the same space as axis-0 (row) of the array indices of pixels in `re_image.tif`.
- `mask.png`：Tissue segmentation based on the valid sequencing area.
- `gene_names.txt`: Gene list.
- `pixel-size-raw.txt`：Side length (in micrometers) of pixels in `re_image.tif`.This value is usually between 0.1 and 1.0.
- `radius-raw.txt`：Number of pixels per spot radius in `re_image.tif`.


In [2]:
import os
import pandas as pd
import cv2
import numpy as np
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2, 40).__str__()

def pad_to_multiple_of_224(img, fill_value=255):
    H, W = img.shape[:2]
    new_H = ((H + 223) // 224) * 224
    new_W = ((W + 223) // 224) * 224

    pad_H = new_H - H
    pad_W = new_W - W

    img_padded = np.pad(
        img,
        ((0, pad_H), (0, pad_W), (0, 0)),
        mode='constant',
        constant_values=fill_value
    )
    return img_padded


he = cv2.imread("/home/lixiaoyu/VISD/xenium-data/Raw-data/Xenium_V1_hLiver_nondiseased_section_FFPE_he_image.ome.tif")
ome = cv2.imread("/home/lixiaoyu/VISD/xenium-data/Raw-data/morphology.ome.tif")
M = pd.read_csv("/home/lixiaoyu/VISD/xenium-data/Raw-data/Xenium_V1_hLiver_nondiseased_section_FFPE_he_imagealignment.csv", header=None).values


scale = 0.2125 / 0.5
new_width = int(he.shape[1] * scale)
new_height = int(he.shape[0] * scale)
he_small = cv2.resize(he, (new_width, new_height), interpolation=cv2.INTER_AREA)
S = np.array([[scale, 0, 0], [0, scale, 0], [0, 0, 1]])
M_new = M.copy()
M_new[0, 2] *= scale  
M_new[1, 2] *= scale  
dst_width = int(ome.shape[1] * scale)
dst_height = int(ome.shape[0] * scale)
img = cv2.warpPerspective(he_small, M_new, dsize=(dst_width, dst_height))
img = img.astype(np.uint8)
img = pad_to_multiple_of_224(img)
cv2.imwrite("/home/lixiaoyu/VISD/xenium-data/Pre-data/re_image.tif", img)

print("✓ Image aligned, padded, and saved.")

✓ Image aligned, padded, and saved.


In [6]:
import cv2
import numpy as np
import scanpy as sc
import pandas as pd
from shapely.geometry import Polygon
import pickle
from skimage.draw import polygon2mask
from tqdm import tqdm

PATHS_Healthy = {
    "matrix_dir": "/home/lixiaoyu/VISD/xenium-data/Raw-data/cell_feature_matrix",
    "boundaries_csv": "/home/lixiaoyu/VISD/xenium-data/Raw-data/cell_boundaries.csv",
    "cells_csv": "/home/lixiaoyu/VISD/xenium-data/Raw-data/cells.csv",
    "gene_names_txt": "/home/lixiaoyu/VISD/xenium-data/Pre-data/gene_names.txt",
    "he_image": "/home/lixiaoyu/VISD/xenium-data/Pre-data/re_image.tif",
    "output_pkl": "/home/lixiaoyu/VISD/xenium-data/Pre-data/genes_3D.pkl"
}


def Cal_area_2poly(data1, data2):
    poly1 = Polygon(data1).convex_hull
    poly2 = Polygon(data2).convex_hull
    if not poly1.intersects(poly2):
        return 0
    return poly1.intersection(poly2).area

def load_dataset(paths):
    adata = sc.read_10x_mtx(paths["matrix_dir"], var_names='gene_symbols')
    adata.var_names_make_unique()
    gene_names = list(adata.var_names)
    counts = np.array(adata.X.todense())

    boundaries = pd.read_csv(paths["boundaries_csv"])
    cells = pd.read_csv(paths["cells_csv"])

    all_cell_ids = pd.concat([cells['cell_id'], boundaries['cell_id']]).unique()
    cell_id_to_idx = {cell_id: idx for idx, cell_id in enumerate(all_cell_ids)}
    cells['cell_id'] = cells['cell_id'].map(cell_id_to_idx)
    boundaries['cell_id'] = boundaries['cell_id'].map(cell_id_to_idx)

    boundaries['vertex_x'] *= 2
    boundaries['vertex_y'] *= 2

    image = cv2.imread(paths["he_image"])
    return adata, counts, gene_names, boundaries, cells, image

def build_cnts(counts, boundaries, cells, image_shape, gene_indices, grid_size=16):
    cnts = np.zeros((image_shape[0] // grid_size, image_shape[1] // grid_size, len(gene_indices)))
    for c in tqdm(range(len(cells)), desc="Processing cells"):
        data1 = boundaries[boundaries['cell_id'] == c][['vertex_x', 'vertex_y']].values

        pixes = (data1 // grid_size * grid_size).astype(int)
        pixes = np.unique(pixes, axis=0)

        min_x, min_y = np.min(data1, axis=0).astype(int) // grid_size * grid_size
        max_x, max_y = np.max(data1, axis=0).astype(int) // grid_size * grid_size

        all_pixes = np.array([[x, y] for x in range(min_x, max_x + grid_size, grid_size)
                                      for y in range(min_y, max_y + grid_size, grid_size)])

        mask = polygon2mask(
            ((max_y - min_y) // grid_size + 1, (max_x - min_x) // grid_size + 1),
            (data1 - [min_x, min_y]) / grid_size
        )

        valid_mask = mask[
            (all_pixes[:, 1] // grid_size - min_y // grid_size),
            (all_pixes[:, 0] // grid_size - min_x // grid_size)
        ]
        valid_pixes = all_pixes[valid_mask]
        pixes = np.unique(np.vstack((pixes, valid_pixes)), axis=0)

        areas = []
        for p in pixes:
            x, y = p
            data2 = np.array([[x, y], [x, y+grid_size], [x+grid_size, y+grid_size], [x+grid_size, y]])
            areas.append(Cal_area_2poly(data1, data2))

        area1 = sum(areas)
        for i, p in enumerate(pixes):
            x, y = p
            h, w = y // grid_size, x // grid_size
            if h >= cnts.shape[0] or w >= cnts.shape[1]:
                continue
            cnts[h, w, :] += counts[c, gene_indices] * (areas[i] / area1)
    return cnts

adata_healthy, counts_healthy, genes_healthy, boundaries_healthy, cells_healthy, image_healthy = load_dataset(PATHS_Healthy)

common_genes = sorted(set(genes_healthy))
idx_healthy = [adata_healthy.var_names.get_loc(g) for g in common_genes]

pd.Series(common_genes).to_csv(PATHS_Healthy["gene_names_txt"], index=False, header=False)

cnts_healthy = build_cnts(counts_healthy, boundaries_healthy, cells_healthy, image_healthy.shape, idx_healthy)
with open(PATHS_Healthy["output_pkl"], 'wb') as f:
    pickle.dump(cnts_healthy, f)

print("✓ Xenium 3D gene tensor has been saved to:", PATHS_Healthy["output_pkl"])

Processing cells: 100%|██████████| 239271/239271 [17:42<00:00, 225.18it/s]


✓ Xenium 3D gene tensor has been saved to: /home/lixiaoyu/VISD/xenium-data/Pre-data/genes_3D.pkl


In [7]:
from scipy.ndimage import binary_dilation
import numpy as np
from matplotlib import pyplot as plt
import pickle


with open("/home/lixiaoyu/VISD/xenium-data/Pre-data/genes_3D.pkl", 'rb') as f:
    gene = pickle.load(f)
    matrix = gene

mask = np.any(matrix, axis=2)
# mask[:70, :] = 0
mask = mask.astype(np.uint8) * 255

bool_mask = mask == 255
dilated_mask = binary_dilation(bool_mask)
new_mask = dilated_mask.astype(np.uint8) * 255

plt.imsave("/home/lixiaoyu/VISD/xenium-data/Pre-data/mask.png", mask, cmap='gray')


In [8]:
import numpy as np
import cv2
import pandas as pd
from utils import load_image

distance = 100  
pixel_size = 0.5
he = cv2.imread("/home/lixiaoyu/VISD/xenium-data/Pre-data/re_image.tif")
list = []
for i in range(0, int(he.shape[1] * pixel_size - 27.5), distance):
    if i/100 % 2 == 0:
        k = 0
    else:
        k = 50
    for j in range(0, int(he.shape[0] * pixel_size - 27.5 - (127.5-k)), distance):
        x = i + 27.5
        y = j + 27.5 + k
        list.append([x, y])
list = np.array(list).astype(np.float32)
list = list // pixel_size  

df = pd.DataFrame(list, columns=['x', 'y'])


mask = load_image("/home/lixiaoyu/VISD/xenium-data/Pre-data/mask.png") > 0
mask = mask[:, :, 0]

rows_to_delete = []
H, W = mask.shape
for i in range(H):
    for j in range(W):
        if mask[i, j] == False:  
            row_to_delete = df[(df['x'] // 16 - 3 == j) & (df['y'] // 16 - 3 == i)]
            if not row_to_delete.empty:
                rows_to_delete.append(row_to_delete.index[0])

df = df.drop(rows_to_delete)
df.to_csv("/home/lixiaoyu/VISD/xenium-data/Pre-data/pseudo_locs.csv", index=False, float_format='%.2f')

Image loaded from /home/lixiaoyu/VISD/xenium-data/Pre-data/mask.png


In [9]:
import numpy as np
import pickle
import pandas as pd
from PIL import Image
Image.MAX_IMAGE_PIXELS = 300000000

def load_image(filename, verbose=True):
    img = Image.open(filename)
    img = np.array(img)
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[..., :3]  # remove alpha channel
    if verbose:
        print(f'Image loaded from {filename}')
    return img

def load_tsv(filename, index=None):
    index_col = 0 if index else None
    df = pd.read_csv(filename, header=0, index_col=index_col)
    print(f'Dataframe loaded from {filename}')
    return df

def get_locs(prefix, target_shape=None):
    df = load_tsv(f'{prefix}pseudo_locs.csv')
    orig_locs = np.stack([df['x'], df['y']], -1)

    used_locs = orig_locs.copy()
    if target_shape is not None:
        wsi = load_image(f'{prefix}re_image.tif')
        current_shape = np.array(wsi.shape[:2])
        rescale_factor = current_shape // target_shape
        used_locs = used_locs.astype(float)
        used_locs /= rescale_factor
        used_locs = used_locs.round().astype(int)

    return orig_locs, used_locs

def get_disk_mask(radius, boundary_width=None):
    radius_ceil = int(np.ceil(radius))
    locs = np.meshgrid(
        np.arange(-radius_ceil, radius_ceil + 1),
        np.arange(-radius_ceil, radius_ceil + 1),
        indexing='ij')
    locs = np.stack(locs, -1)
    distsq = (locs**2).sum(-1)
    isin = distsq <= radius**2
    if boundary_width is not None:
        isin &= distsq >= (radius - boundary_width)**2
    return isin

def get_patches_flat(img, used_locs, mask, orig_locs):
    shape = np.array(mask.shape)
    center = shape // 2
    r = np.stack([-center, shape - center], -1)
    x_list = []
    valid_orig_locs = []
    skipped = 0

    for idx, s in enumerate(used_locs):
        y1 = s[1] + r[0][0]
        y2 = s[1] + r[0][1]
        x1 = s[0] + r[1][0]
        x2 = s[0] + r[1][1]

        if y1 < 0 or y2 > img.shape[0] or x1 < 0 or x2 > img.shape[1]:
            skipped += 1
            continue

        patch = img[y1:y2, x1:x2]

        if patch.shape[:2] != mask.shape:
            skipped += 1
            continue

        x = patch[mask]  # shape: (n_pixels, genes)
        x_list.append(x)
        valid_orig_locs.append(orig_locs[idx])

    print(f"[INFO] Skipped {skipped} patches due to boundary issues.")
    x_list = np.stack(x_list)
    valid_orig_locs = np.array(valid_orig_locs)
    return x_list, valid_orig_locs



prefix = "/home/lixiaoyu/VISD/xenium-data/Pre-data/"

with open(prefix + "genes_3D.pkl", 'rb') as f:
    img = pickle.load(f)

radius = 55 / 16
mask = get_disk_mask(radius)


orig_locs, used_locs = get_locs(prefix, np.array([img.shape[0], img.shape[1]]))

x, filtered_orig_locs = get_patches_flat(img, used_locs, mask, orig_locs)

cnts = np.sum(x, axis=1)


with open(prefix + "gene_names.txt", 'r') as file:
    gene_names = [line.strip() for line in file]

assert cnts.shape[1] == len(gene_names), "Mismatch between gene count and gene_names"

cnts_df = pd.DataFrame(data=cnts, columns=gene_names)
cnts_df.to_csv(prefix + "pseudo_st.csv", index=False)

filtered_df = pd.DataFrame(filtered_orig_locs, columns=["x", "y"])
filtered_df.to_csv(prefix + "pseudo_locs.csv", index=False)

print("[DONE] Saved pseudo_st.csv and pseudo_locs.csv.")


Dataframe loaded from /home/lixiaoyu/VISD/xenium-data/Pre-data/pseudo_locs.csv


Image.py (3368): Image size (366937088 pixels) exceeds limit of 300000000 pixels, could be decompression bomb DOS attack.


Image loaded from /home/lixiaoyu/VISD/xenium-data/Pre-data/re_image.tif
[INFO] Skipped 0 patches due to boundary issues.
[DONE] Saved pseudo_st.csv and pseudo_locs.csv.
